In [1]:
import pandas as pd
import json
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import operator
from tqdm import tqdm
import os

In [2]:
from text_processing import get_text_map


C:\Users\Asus\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [11]:
USER_NAME = "big_musician"

In [12]:
texts_lenta = pd.read_csv("./articles/music_lenta.csv")
texts_lenta.iloc[16]['text']

"Австралийские электронные музыканты The Avalanches выпустили видео на песню Because I'm Me. Клип, снятый режиссером Грегом Бранкаллой, рассказывает о мальчике, который влюблен в работницу метрополитена и пытается привлечь ее внимание при помощи песни. Because I'm Me — четвертый сингл с альбома The Avalanches 2016 года Wildflower, который стал их первой пластинкой за 16 лет."

In [13]:
text_db = pd.read_csv(USER_NAME + "_text_db.csv",header = None)
text_db.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.6118,0.75,0.10,0.166000,0.200000,0.200000,0.024267,0.030002,0.000000,0.071312,0.191511,1.0
1,0.8013,0.63,0.14,0.180714,0.000000,0.071429,0.015801,0.032847,0.004245,0.145502,0.212638,1.0
2,0.5788,0.70,0.15,0.158667,0.333333,0.266667,0.031384,0.032255,0.000000,0.130564,0.209426,1.0
3,0.4068,0.55,0.33,0.141818,0.242424,0.303030,0.021535,0.010711,0.000000,0.054207,0.195904,1.0
4,0.6818,0.68,0.11,0.155455,0.000000,0.090909,0.013307,0.019474,0.000000,0.062778,0.191883,1.0


In [14]:
sent_db = pd.read_csv(USER_NAME + "_sentence_db.csv",header = None)
sent_db.head()

,0,1,2,3,4,5,6,7
0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.209091,1.0
1,0.018519,0.000000,0.021353,0.000000,0.0,0.042707,0.347170,1.0
2,0.034483,0.034483,0.000000,0.063484,0.0,0.143760,0.217857,1.0
3,0.000000,0.000000,0.000000,0.000000,0.0,0.177674,0.219048,1.0
4,0.000000,0.000000,0.000000,0.000000,0.0,0.107480,0.205263,1.0


In [15]:
word_db = pd.read_csv(USER_NAME +  "_word_db.csv",header = None)
word_db.head()

,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,300,301,302
0,72.0,155.10,0.019617,0.027856,-0.098140,0.018417,-0.060545,0.052610,-0.008416,0.019377,...,-0.045566,-0.019588,-0.051453,-0.000417,0.015935,0.018377,-0.049183,0.040319,0.087135,1.0
1,1.0,17.10,-0.059523,0.012793,0.034546,-0.019653,-0.059405,-0.108276,-0.020982,0.038993,...,0.080227,0.072730,0.008212,0.018211,-0.058539,-0.046240,-0.067024,-0.060923,-0.018587,1.0
2,73.0,483.30,-0.012936,-0.003571,-0.036513,-0.125645,-0.025398,-0.118884,-0.007071,-0.008625,...,0.069345,0.021790,-0.002086,0.007569,-0.078134,0.048388,-0.009063,0.001203,-0.019108,1.0
3,71.0,16049.95,0.052736,0.012474,-0.023544,0.048177,-0.050497,-0.054198,0.043881,-0.023119,...,0.015864,0.024000,0.093944,0.019959,-0.034256,-0.002028,-0.055652,-0.034802,-0.000229,1.0
4,81.0,1129.20,-0.041698,-0.001450,0.043212,0.008522,0.035688,-0.040512,0.000922,0.040212,...,0.005199,0.017304,0.067342,-0.028293,-0.010177,-0.018577,-0.000790,0.050796,-0.052772,1.0


with open ("text_map_improved_example.json", "r", encoding = "utf-8") as f:
    examined_text = json.load(f)

In [16]:
words_dict = []
with open("colloc_lit.txt", "r", encoding = "utf-8") as f:
    for line in f.readlines():
        words_dict.append(line[:-1])

In [ ]:
words_dict[0]

In [ ]:
my_list = [ ('John', 40), ('Dana', 35), ('Betty', 10), ('Robby', 8), ('John', 20) ]
my_list.sort(key = operator.itemgetter(1))
print(my_list)

In [17]:
def get_top_similar_vectors_from_marked_db(analyzed_text_map,marked_text_db, marked_sent_db, marked_word_db, words_indexed_dict):
    
    similarity_json = {"similar_texts":[],"simiar_sentences":[], "similar_collocations":[]}
    #text_similarity
    text_vector = []
    text_vector.append(analyzed_text_map['lix'])
    text_vector.append(analyzed_text_map['ttr'])
    text_vector.append(analyzed_text_map['sentences_count'])
    text_vector.append(analyzed_text_map['average_sentence_length'])
    text_vector.extend(analyzed_text_map['sent_properties'])
    list_of_texts_similarities = []
    for text_ind in range(len(marked_text_db)):
        #print(text_vector)
        #target_var = list(marked_text_db.iloc[2][-1:]) WILL NEED WHEN PREDICT!
        v1 = np.array(text_vector).reshape(1, -1)
        v2 = np.array(list(marked_text_db.iloc[text_ind][:-1])).reshape(1, -1)
        sim = cosine_similarity(v1,v2)
        #print(sim[0][0])
        if sim[0][0] > 0.6:
            list_of_texts_similarities.append((sim[0][0], {"TEXT_IND_in_marked_db":text_ind, "VECTOR":list(marked_text_db.iloc[text_ind][:-1])}))
            
    list_of_texts_similarities.sort(key = operator.itemgetter(0), reverse= True)   
    similarity_json['similar_texts'] = list_of_texts_similarities[:5]
    
    list_of_all_sentences_similarity = []
    for sentence in analyzed_text_map['sentences_map']:
        current_sentence = []
        current_sentence.append(sentence['spec_sentence_features']['negation'])
        current_sentence.append(sentence['spec_sentence_features']['coreference'])
        current_sentence.append(sentence['spec_sentence_features']['vozvr_verb'])
        current_sentence.append(sentence['spec_sentence_features']['prich'])
        current_sentence.append(sentence['spec_sentence_features']['deepr'])
        current_sentence.append(sentence['spec_sentence_features']['case_complexity'])
        current_sentence.append(sentence['spec_sentence_features']['mean_depend_length'])
        v1 = np.array(current_sentence).reshape(1, -1)
        list_of_current_sent_similarities = []
        for sent_ind in range(len(marked_sent_db)):
            v2 = np.array(list(marked_sent_db.iloc[sent_ind][:-1])).reshape(1, -1)
            sim = cosine_similarity(v1,v2)
            if sim[0][0] > 0.6:
                list_of_current_sent_similarities.append((sim[0][0], {"sent_ind_in_marked_db":sent_ind, "sent_vector":list(marked_sent_db.iloc[sent_ind][:-1])}))
        list_of_current_sent_similarities.sort(key = operator.itemgetter(0), reverse= True)
        list_of_all_sentences_similarity.append(list_of_current_sent_similarities[:5])
    similarity_json['simiar_sentences'] = list_of_all_sentences_similarity
    
    list_of_all_words_similarity = []
    for sentence in analyzed_text_map['sentences_map']:
        for colloc in sentence['collocation_vectors_list']:
            current_word_similarity = []
            #print(colloc[1][0])
            freq_vec = np.array([colloc[1][1], colloc[1][2]]).reshape(1,-1)
            colloc_w2v = np.array(colloc[2]).reshape(1,-1)
            for db_word_index in range(len(marked_word_db)):
                compared_raw_word_vect = marked_word_db.iloc[db_word_index][:-1]
                
                comp_word2vec = compared_raw_word_vect[2:]
                #print(db_word_index, len(comp_word2vec))
                comp_colloc_w2v = np.array(comp_word2vec).reshape(1,-1)
                w2v_sim = cosine_similarity(colloc_w2v,comp_colloc_w2v)
                #print("w2v", sim)
                
               
                comp_freq_vec = np.array([compared_raw_word_vect[0], compared_raw_word_vect[1]]).reshape(1,-1)
                sim_freq = cosine_similarity(freq_vec,comp_freq_vec)
                #print("freq", sim_freq)
                
                w2v_sim = w2v_sim[0][0]
                sim_freq = sim_freq[0][0]
                
                average_similarity = w2v_sim*0.8 + sim_freq*0.2
                
                word_comp_json = {"colloc_db_index":db_word_index, "sim_colloc":words_indexed_dict[db_word_index],"w2v_sim":w2v_sim,"freqence_similarity":sim_freq}
                if average_similarity> 0.6:
                    current_word_similarity.append((word_comp_json,average_similarity))
            current_word_similarity.sort(key = operator.itemgetter(1), reverse= True)
            list_of_all_words_similarity.append((colloc[1][0],current_word_similarity[:5]))
    similarity_json['similar_collocations'] = list_of_all_words_similarity
    return similarity_json

In [ ]:
examined_text = get_text_map(texts_lenta.iloc[69]['text'], raw_text_input=True)
text_sim_list = get_top_similar_vectors_from_marked_db(examined_text,text_db,sent_db,word_db,words_dict)

In [ ]:
text_sim_list

In [ ]:
handel_texts_index = ['0',
 '17',
 '26',
 '32',
 '98',
 '121',
 '130',
 '133',
 '231',
 '240',
 '316',
 '331',
 '334',
 '336',
 '366',
 '371']
for text_ind in tqdm(range(len(texts_lenta))):
    if str(text_ind) not in handel_texts_index:
        examined_text = get_text_map(texts_lenta.iloc[text_ind]['text'], raw_text_input=True)
        text_sim_list = get_top_similar_vectors_from_marked_db(examined_text,text_db,sent_db,word_db,words_dict)
        with open(os.path.join('./music_similarity',str(text_ind) + ".json"), "w", encoding = 'utf-8') as f:
            json.dump(text_sim_list, f, indent = 2, ensure_ascii=False)


  4%|██▋                                                                     | 193/5099 [8:01:47<204:07:03, 149.78s/it]

In [ ]:
len(text_sim_list['similar_texts'])

In [ ]:
len(text_sim_list['simiar_sentences'][1])

In [ ]:
text_sim_list['simiar_sentences']

In [ ]:
for sim in text_sim_list['similar_collocations']:
    print (sim)
    print()

# В ПРОЦЕСЕ РЕКОММЕНДАЦИИ ПОЛЬЗОВАТЬСЯ ИНДЕКСАМИ РАЗМЕЧЕННЫХ ПРЕДЛОЖЕНИЙ ИЗ БАЗЫ ДАННЫХ
ТО ЕСТЬ В КАРТАХ ИМИЛАРИТИ ДОЛЖНЫ БЫТЬ СРАЗУ ИНДЕКСЫ ВСЕЙ ХЕРНИ

In [ ]:
for text in text_sim_list:
    if text[1] == text_vector:
        print("yes")